# 01. Azure Machine Learning - Environment Setup
This notebook gets a reference to an Azure Machine Learning workspace using the AML SDK and performs a few operations in preparation for this demo. 
 - Create a compute cluster to be used for model training
 - Upload a sample time-series to an AML-linked blob datastore. 
 - Create a registered dataset that can be used as an input to an AutoML Forecasting job
 
Run all of the cells in this notebook to setup the necessary components for the ML experiment.

### Import required packages

In [ ]:
from azureml.core import Workspace, Datastore, Dataset
from azureml.data import DataType
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

### Connect to AML workspace

In [ ]:
ws = Workspace.from_config()

### Create Azure ML Compute Cluster for model training
Compute clusters scalable, on-demand, compute resources that can be dynamically spun up and down to support different model training and inferencing jobs - think of compute clusters as massively scalable, serverless, ML compute. Here, we are creating a GPU cluster that can provision up to 2 nodes depending upon the workload submitted - this upper limit can be increased even futher. Once jobs complete and nodes sit idle for 120 seconds they are automatically spun down and billing stops.

[Create an Azure Machine Learning Compute Cluster](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-attach-compute-cluster?tabs=python)

[What are Compute Targets in Azure Machine Learning](https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target)

In [ ]:
cluster_name = 'cpucluster'
# compute_target = ws.compute_targets[cluster_name]
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing compute target.")
except ComputeTargetException:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_DS3_V2',
        idle_seconds_before_scaledown=120,
        min_nodes=0,
        max_nodes=2,
    )
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    # Can poll for a minimum number of nodes and for a specific timeout.
    # If no min_node_count is provided, it will use the scale settings for the cluster.
    compute_target.wait_for_completion(
        show_output=True, min_node_count=None, timeout_in_minutes=20
    )
    

### Get pointer to default Azure ML Blob Datastore
Datastores represent storage locations that are attached to an Azure Machine Learning Workspace. Connection information for these storage services are stored in an AML-linked Key Vault and they can be accessed either through the AML UI or SDK.

Azure Machine Learning workspaces are provisioned with an attached Azure Storage Account by default. These default blobstores are generally used for storing artifacts from ML experiments like trained models, saved outputs, run logs etc. Here we retrieve a reference to the default datastore.

[Connect to Storage Services on Azure Machine Learning with Datastores](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-access-data)

In [ ]:
default_ds = ws.get_default_datastore()

### Create new datastore and upload sample data
Azure Blob Storage containers can be registered as datastores in an AML workspace. For the purposes of our demonstration, we are going to create a separate blob container (in the attached default storage account) specifically for sample data storage, and register this as a new datastore. <i>Note:</i> This is purely for demonstration purposes, images could be loaded from any target datastore for model training.

[Creating a New Azure Machine Learning Datastore]('https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.datastore.datastore?view=azure-ml-py')

In [ ]:
new_container_name = 'timeseriesdatastore'
default_ds.blob_service.create_container(new_container_name)

try:
    sample_datastore = Datastore.get(ws, new_container_name)
    print("Found Blob Datastore with name: %s" % new_container_name)
except Exception:
    imagestore = Datastore.register_azure_blob_container(
     workspace=ws,
        datastore_name = new_container_name, 
        account_name = default_ds.account_name,
        container_name = new_container_name,
        account_key = default_ds.account_key)
    print("Registered blob datastore with name: %s" % new_container_name)
    
# Upload images to new datastore
sample_datastore.upload('./sample_data', 'sample_data', overwrite=True, show_progress=True)

### Create Dataset in the Azure Machine Learning workspace
To train a custom time-series forecasting model we need to provide a dataset to the AutoML job. For the purposes of our demo we're utilizing sample data from the Azure Open Dataset 'OJ Sample Data'. The code below will create a registered dataset named `Sample_TimeSeries` from the sample CSV file uploaded to the datastore.

In [ ]:
sample_dataset = Dataset.Tabular.from_delimited_files(path=(sample_datastore, 'sample_data/*'))
sample_dataset.register(ws, 'Sample_TimeSeries', create_new_version=True)